In [1]:
#Loading the libraries
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout   
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras import optimizers

import tensorflow as tf
from tensorflow.contrib import rnn


# Reading csv file
#customers timeseries updated has data for 100 customers
df = pd.read_csv("customers_timeseries_updated.csv")
#only using required columns for modelling
df = df[['client_debtor_number','dates','fv_cost']]
# number of unique customers in the data
print("number of unique customers",len(df['client_debtor_number'].unique()))

# subsetting one customer data from the set
# 1015193,7370830
df_1015130 = df[df['client_debtor_number'] == 1015193]
del df_1015130['client_debtor_number']


Using TensorFlow backend.
/Users/anusha/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


number of unique customers 101


In [2]:
#weekly predictions
df_1015130['dates'] = pd.to_datetime(df_1015130['dates'],format= '%d/%m/%Y')
#weekly data
week_data = df_1015130.set_index('dates').resample('1W').mean()
#week_data['dates'] = pd.to_datetime(week_data['dates'], errors='coerce')
week_data['dates'] = week_data.index.values
#week_data = week_data.resample('1W')

/Users/anusha/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [29]:
len(df_1015130)

3478

In [3]:
week_data.head()

,fv_cost,dates
dates,,
2010-01-03,0.0,2010-01-03
2010-01-10,0.0,2010-01-10
2010-01-17,0.0,2010-01-17
2010-01-24,0.0,2010-01-24
2010-01-31,0.0,2010-01-31


In [4]:
# splitting train and test datasets 
#training 2010-2017, testing on 2018 data\
train_x = week_data[~(week_data['dates'].dt.year == 2018)]
test_x = week_data[week_data['dates'].dt.year == 2018]
#resetting index values
train_x.reset_index(drop=True,inplace=True)
test_x.reset_index(drop=True,inplace=True)

#sorting data based on dates
train_x['dates'] = pd.to_datetime(train_x['dates'],format= '%d/%m/%Y')
train_x.sort_values(by='dates',inplace=True)
test_x['dates'] = pd.to_datetime(test_x['dates'],format= '%d/%m/%Y')
test_x.sort_values(by='dates',inplace=True)

/Users/anusha/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/anusha/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/anusha/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [5]:
# lag_feature function is used to get new column in the dataframe with 
#lagged data,number of lags can be given as function parameter 
def lag_feature(df, lag=1):
    if not type(df) == pd.DataFrame:
        df = pd.DataFrame(df, columns=['fv_cost'])   
    def rename_lag(ser, j):
        ser.name = ser.name + f'_{j}'
        return ser        
    # add a column lagged by `i` steps
    for i in range(1, lag + 1):
        df = df.join(df.fv_cost.shift(i).pipe(rename_lag, i))
    df.dropna(inplace=True)
    return df

# Prepare training data function is used to scale the fv_cost values 
# between -1 to 1 and calls lag_feature to create lag columns 
def prepare_training_data(series_data, lag):
    " Converts a series of data into a lagged, scaled sample."
    # scale training data
    scaler = MinMaxScaler(feature_range=(-1, 1))
    #when diff is used don't use values
    #cost_vals = scaler.fit_transform(series_data.values.reshape(-1, 1))   
    cost_vals = scaler.fit_transform(series_data.values.reshape(-1, 1))
    # convert series to lagged features
    cost_lagged = lag_feature(cost_vals, lag=lag)
    # X, y format taking the first column (original time series) to be the y
    X = cost_lagged.drop('fv_cost', axis=1).values
    y = cost_lagged.fv_cost.values
    
    # keras expects 3 dimensional X
    X = X.reshape(X.shape[0], 1, X.shape[1])   
    return X, y, scaler

def prepare_training_data_tensorflow(series_data, lag):
    " Converts a series of data into a lagged, scaled sample."
    # scale training data
    scaler = MinMaxScaler(feature_range=(-1, 1))
    #when diff is used don't use values
    #cost_vals = scaler.fit_transform(series_data.values.reshape(-1, 1))   
    cost_vals = scaler.fit_transform(series_data.values.reshape(-1, 1))
    # convert series to lagged features
    cost_lagged = lag_feature(cost_vals, lag=lag)
    # X, y format taking the first column (original time series) to be the y
    X = cost_lagged.drop('fv_cost', axis=1).values
    y = cost_lagged.fv_cost.values
    
    # keras expects 3 dimensional X
    X = X.reshape(X.shape[0], X.shape[1], 1)   
    return X, y, scaler

#Difference function is get lag difference values
def difference(dataset):
    interval=1
    diff = list()
    for i in range(interval, len(dataset)):
        #print ("iteration ",i)
        value = (dataset.iloc[i] - dataset.iloc[i - interval])/(1+abs(dataset.iloc[i - interval]))
        diff.append(value)
    return diff

# if you are using difference lag values use this code
#diff_train = np.array(difference(train_x['fv_cost']))
#diff_test = np.array(difference(test_x['fv_cost']))
#train_x_cust,train_y_cust, scalar_train = prepare_training_data(diff_train, 3)
#test_x_cust,test_y_cust,scalar_test = prepare_training_data(diff_test, 3)
# preparing data on train and test fv_cost column values
train_x_cust,train_y_cust, scalar_train = prepare_training_data(train_x['fv_cost'], 3)
test_x_cust,test_y_cust,scalar_test = prepare_training_data(test_x['fv_cost'], 3)
print(train_x_cust.shape)
print(train_y_cust.shape)

(415, 1, 3)
(415,)


In [6]:
# Predictions for one timestep
def lstm_model(train_x, train_y,lag=3,num_neurons=50):
    # Model description
    # lag is number of lags used to prepare the data
    #lag =  3
    # model parameters
    #num_neurons = 50 #number of neurons/nodes for the layer
    # actually we can give data in batches to the model
    # number of samples in the data should be divisible by batch size
    # we are giving all data for a customer as one batch
    batch_size = 1  
    # input_shape as required by LSTM layer
    batch_input_shape=(batch_size, 1, lag)
    # dropout rate used in dropout layer
    dropout_rate =0.2
    # instantiate a sequential model
    model = Sequential()
    #add convolution layer
    # input_shape=(3,1) 3 lags columns and 1 y value
    # when strides>1 you cannot have dilation>1
    # activation_func options ['softmax', 'softplus', 'softsign', 'relu', 
    #                   'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
    # in conv1D layer we can modify filters,kernel_size,strides and activation
    # parameters
    model.add(Conv1D(filters=num_neurons,batch_size=1, kernel_size=3, strides=3, 
                 padding="same",activation='linear',dilation_rate=1, 
                 input_shape=(1, 3),data_format='channels_first'))
    # maxpooling layers tries to reduce the features by taking maximum value for
    # window/pool size selected,strides and pool_size can be changed
    model.add(MaxPooling1D(pool_size=3,strides=3, padding="same"))
    model.add(Dropout(dropout_rate))
    # add LSTM layer - stateful MUST be true here in 
    # order to learn the patterns within a series
    model.add(LSTM(units=num_neurons, 
              batch_input_shape=batch_input_shape, return_sequences=False,# as we only want last hidden output 
              stateful=True))
    # output layer
    model.add(Dense(1,activation='linear'))
    # compile the model with required loss and optimizer function
    adam = optimizers.Adam(lr=0.01, decay=0.01)
    model.compile(loss='mean_squared_error', optimizer=adam)
    model.fit(train_x, train_y, epochs=10, batch_size=batch_size, verbose=2, shuffle=False)
    return model

In [7]:
model = lstm_model(train_x_cust,train_y_cust,3,50)

Epoch 1/10
 - 7s - loss: 0.0441
Epoch 2/10
 - 5s - loss: 0.0671
Epoch 3/10
 - 5s - loss: 0.0218
Epoch 4/10
 - 5s - loss: 0.0171
Epoch 5/10
 - 5s - loss: 0.0133
Epoch 6/10
 - 5s - loss: 0.0126
Epoch 7/10
 - 5s - loss: 0.0115
Epoch 8/10
 - 5s - loss: 0.0116
Epoch 9/10
 - 5s - loss: 0.0106
Epoch 10/10
 - 5s - loss: 0.0094


In [8]:
n_features = 3
graph = tf.Graph()
with graph.as_default():
    xplaceholder = tf.placeholder(tf.float32,[None,n_features,1])
    yplaceholder = tf.placeholder(tf.float32,[None, n_features,1])

In [9]:
# Training Parameters
learning_rate = 0.001
training_steps = 10000
batch_size = 128
display_step = 200

timesteps = 1
n_output = 1
n_hidden = 20 # number of hidden neurons
# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_output]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_output]))
}

In [10]:
def RNN(x, weights, biases):
    n_features = 3
    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    #x = tf.unstack(x, 3, 1)
    x = tf.unstack(x, 3, 1)
    #x = tf.split(x, n_features, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = tf.nn.rnn_cell.LSTMCell(n_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    #outputs, states = tf.nn.dynamic_rnn(lstm_cell, x, dtype=tf.float32)
    
    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [11]:
logits = RNN(xplaceholder, weights, biases)

ValueError: Tensor("rnn/lstm_cell/kernel:0", shape=(21, 80), dtype=float32_ref) must be from the same graph as Tensor("concat:0", shape=(?, 21), dtype=float32).

num_units = 50
batch_size =1
conv_layer = tf.contrib.rnn.Conv1DLSTMCell(num_units,input_shape=[1, 3],
                                           kernel_shape=3,output_channels=1,
                                          use_bias=True,forget_bias=1.0)#stride=3,
seq_length = tf.placeholder(tf.int32, [None])
#val,state = tf.nn.dynamic_rnn(cell=conv_layer, inputs=train_x_cust, sequence_length=seq_length, dtype=tf.float64)
#val,state = tf.nn.dynamic_rnn(cell=conv_layer, inputs=xplaceholder, sequence_length=seq_length, dtype=tf.float32)
initial_state = conv_layer.zero_state(batch_size, dtype=tf.float32)
val, state = tf.nn.dynamic_rnn(conv_layer, tf.expand_dims(xplaceholder, -1), initial_state=initial_state,dtype = tf.float32)
val = tf.transpose(val, [1, 0, 2]) #axis [1,0,2] horizontal and vertical axes
# are interchanged for 2D image;0-horizontal,1-vertical
last = tf.gather(val, int(val.get_shape()[0]) - 1)
weight = tf.Variable(tf.truncated_normal([num_hidden, int(yplaceholder.get_shape()[1])]))
bias = tf.Variable(tf.constant(0.1, shape=[yplaceholder.get_shape()[1]]))


In [12]:
tf.reset_default_graph()
num_neurons = 25 #filters
with graph.as_default():
# other way of implementation
    x = tf.unstack(xplaceholder, 3, 1)
    dropout_prob = tf.placeholder(tf.float32)
    conv_layer = tf.layers.conv1d(inputs=xplaceholder, filters=num_neurons, kernel_size=3, strides=3,
        padding='same', activation = tf.nn.relu) #can change activation
    #lstm_cell = tf.nn.rnn_cell.LSTMCell(n_hidden, forget_bias=1.0)
    #pool = tf.nn.pool(lstm_cell, [3], 'MAX', 'SAME', strides = [3])
    pool_1 = tf.layers.max_pooling1d(conv_layer, pool_size=3, strides=3, padding='SAME')
    drop_out = tf.nn.dropout(pool_1, dropout_prob)
    #dropout_wrapper = tf.contrib.rnn.DropoutWrapper(pool_1,input_keep_prob=(1-dropout_prob))
    #lstm_cell = tf.contrib.rnn.LSTMBlockCell(num_units=num_neurons,use_peephole=False)
    #rnn_cell = tf.contrib.rnn.MultiRNNCell([drop_out,lstm_cell]*1,state_is_tuple=False)
    #output_cell = tf.layers.dense(rnn_cell, 1, tf.nn.relu)
    # Get lstm cell output
    outputs, states = tf.contrib.rnn.static_rnn(drop_out, x, dtype=tf.float32)
    
    

TypeError: The argument 'cell' (Tensor("dropout/mul:0", shape=(?, 1, 25), dtype=float32)) is not an RNNCell: 'output_size' property is missing, 'state_size' property is missing, either 'zero_state' or 'get_initial_state' method is required, is not callable.

In [ ]:
rnn_cell = tf.contrib.rnn.MultiRNNCell([
	tf.contrib.rnn.LSTMBlockCell(NUM_HIDDEN) for _ in range(NUM_LAYERS)
])
rnn_out, _ = tf.nn.dynamic_rnn(rnn_cell, embeddings, dtype=tf.float32)


In [ ]:

cell = tf.nn.rnn_cell.LSTMCell(num_units=64, state_is_tuple=True)
cell = tf.nn.rnn_cell.DropoutWrapper(cell=cell, output_keep_prob=0.5)
cell = tf.nn.rnn_cell.MultiRNNCell(cells=[cell] * 4, state_is_tuple=True)

In [ ]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  outputs_val, states_val = sess.run([outputs, states], 
                                     feed_dict={X: X_batch, seq_length: seq_length_batch})


In [25]:
train_x_cust,train_y_cust, scalar_train = prepare_training_data_tensorflow(train_x['fv_cost'], 3)
test_x_cust,test_y_cust,scalar_test = prepare_training_data_tensorflow(test_x['fv_cost'], 3)
print(train_x_cust.shape)
print(test_x_cust.shape)

(415, 3, 1)
(36, 3, 1)


In [23]:
# other approach
tf.reset_default_graph()

num_periods = 3
inputs = 1
output = 1
hidden = 25

X = tf.placeholder(tf.float32,[None,num_periods,inputs])
#y = tf.placeholder(tf.float32,[None,num_periods,output])
y = tf.placeholder(tf.float32,[None])

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units= hidden,activation=tf.nn.relu)
rnn_output,states = tf.nn.dynamic_rnn(basic_cell,X,dtype=tf.float32)

learning_rate = 0.001

stacked_rnn_output = tf.reshape(rnn_output,[-1,hidden])
#stacked_rnn_output = tf.reshape(rnn_output,[-1])
stacked_outputs = tf.layers.dense(stacked_rnn_output, output)
outputs = tf.reshape(stacked_outputs, [-1, num_periods, output])
#outputs = tf.reshape(stacked_outputs, [-1])

loss = tf.reduce_sum(tf.square(outputs - y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()

In [31]:
#Delete all flags before declare

def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:
        FLAGS.__delattr__(keys)

del_all_flags(tf.flags.FLAGS)

In [32]:
tf.flags.DEFINE_integer("batch_size", 37, "Batch size during training") 
tf.flags.DEFINE_integer("eval_batch_size", 37, "Batch size during evaluation")
tf.flags.DEFINE_integer("test_batch_size", 37, "Batch size for testing")
# above batch_size value works fine with value one as well but remember to run
#del_all_flags function before declaring flags
epochs = 1000
with tf.Session() as sess:
     init.run()
     for ep in range(epochs):
         sess.run(training_op, feed_dict = {X:train_x_cust, y:train_y_cust})
         if ep%100 ==0:
            mse = loss.eval(feed_dict = {X:train_x_cust, y:train_y_cust})
            print(ep,"\tMSE:",mse)
     y_pred = sess.run(outputs, feed_dict = {X:test_x_cust})
     print(y_pred)

0 	MSE: 257669.44
100 	MSE: 71424.44
200 	MSE: 68341.62
300 	MSE: 67958.99
400 	MSE: 67757.8
500 	MSE: 67690.94
600 	MSE: 67670.58
700 	MSE: 67661.74
800 	MSE: 67655.78
900 	MSE: 67650.9
[[[-0.53336775]
  [-0.52872825]
  [-0.5240464 ]]

 [[-0.52029777]
  [-0.5212044 ]
  [-0.51584023]]

 [[-0.519748  ]
  [-0.51868016]
  [-0.5187571 ]]

 [[-0.5198285 ]
  [-0.51911056]
  [-0.5192231 ]]

 [[-0.5177237 ]
  [-0.515473  ]
  [-0.51486087]]

 [[-0.5160373 ]
  [-0.5135942 ]
  [-0.5147239 ]]

 [[-0.5170448 ]
  [-0.5139308 ]
  [-0.51153594]]

 [[-0.51793116]
  [-0.51467806]
  [-0.5171018 ]]

 [[-0.51835555]
  [-0.51920295]
  [-0.520518  ]]

 [[-0.5169399 ]
  [-0.5191648 ]
  [-0.5166661 ]]

 [[-0.51837665]
  [-0.5285798 ]
  [-0.52507114]]

 [[-0.51821023]
  [-0.5238525 ]
  [-0.5174793 ]]

 [[-0.51812214]
  [-0.5168929 ]
  [-0.52077025]]

 [[-0.5150303 ]
  [-0.521441  ]
  [-0.5189402 ]]

 [[-0.51663065]
  [-0.5235782 ]
  [-0.52069795]]

 [[-0.5183123 ]
  [-0.52129334]
  [-0.5209099 ]]

 [[-0.5150602

In [16]:
tf.__version__

'1.11.0'

In [26]:
print(test_y_cust)
print(y_pred - test_y_cust)


[-0.76316197 -0.73862634 -0.74221991 -0.64829467 -0.56661377 -0.50677036
 -0.45412357 -0.36753847 -0.27912729 -0.35985935 -0.42056837 -0.31465915
 -0.23698166 -0.18262504 -0.14319251 -0.05493859  0.07575444  0.20911232
  0.32464201  0.40540166  0.51765344  0.63870508  0.36279695  0.38892744
  0.39343426  0.51660125  0.66388579  0.76612264  0.85384363  1.
 -0.28655686 -0.9000785  -0.87877479 -0.79834174 -0.75466256 -0.7067018 ]
[[[0.23436965 0.20983402 0.21342759 ... 0.26954942 0.22587024 0.17790948]
  [0.2411591  0.21662346 0.22021704 ... 0.27633887 0.23265968 0.18469892]
  [0.24155445 0.21701882 0.2206124  ... 0.27673422 0.23305504 0.18509428]]

 [[0.24248405 0.21794842 0.22154199 ... 0.27766382 0.23398463 0.18602387]
  [0.23960562 0.21506999 0.21866356 ... 0.27478539 0.23110621 0.18314545]
  [0.244101   0.21956537 0.22315894 ... 0.27928077 0.23560159 0.18764083]]

 [[0.24358357 0.21904794 0.22264152 ... 0.27876334 0.23508416 0.1871234 ]
  [0.2455637  0.22102807 0.22462164 ... 0.28074